In [1]:
import os
import nibabel as nib
import sys
import tensorflow as tf
import pandas as pd 
import numpy as np
import subprocess
from glob import glob
from tensorflow import keras
from nilearn import image
from sklearn.model_selection import train_test_split

2024-01-29 12:09:22.721119: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-29 12:09:23.886447: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-29 12:09:23.887279: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-01-29 12:09:23.887330: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
def read_nii_metadata(nii_path):
    """Load in nifti data and header information"""
    nii = nib.load(nii_path)
    nii_affine = nii.affine
    nii_data = nii.get_fdata()
    nii_header = nii.header
    return nii_affine, nii_data, nii_header

def normalization_shifted(nii_path):
    aff, nii_img, head = read_nii_metadata(nii_path)
    shifted = nii_img - nii_img.min()
    scaled = (shifted - shifted.min())/(shifted.max()-shifted.min())
    img_nii = nib.Nifti1Image(scaled, aff, head)
    return img_nii

#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6758567/ - Z score normalization
#previous unet paper normalizes with 0 mean and 1 standard deviation
def z_normalization(nii_path):
    aff, nii_img, head = read_nii_metadata(nii_path)
    img_stdev = np.std(nii_img)
    print(img_stdev)
    img_mean = np.mean(nii_img)
    z_norm = (nii_img - img_mean)/img_stdev
    new_img = nib.Nifti1Image(z_norm, aff, head)
    return new_img
    
def min_max_normalization(nii_path):
    aff, nii_img, head = read_nii_metadata(nii_path)
    scaled_image = (nii_img - np.min(nii_img)) / (np.max(nii_img) - np.min(nii_img))
    img_nii = nib.Nifti1Image(scaled_image, aff, head)
    return img_nii

In [5]:
cd

/home/athurai3


In [6]:
test_subject_path = 'scratch/test_noninterp_image/sub-P001/sub-P001_desc-rigid_space-T1w_ct.nii.gz'
norm_shift_path = 'scratch/test_noninterp_image/sub-P001/sub-P001_desc-rigid_shiftnorm_space-T1w_ct.nii.gz'
z_norm_path = 'scratch/test_noninterp_image/sub-P001/sub-P001_desc-rigid_space-T1w_znorm_ct.nii.gz'
min_max_path = 'scratch/test_noninterp_image/sub-P001/sub-P001_desc-rigid_minmax_space-T1w_ct.nii.gz'
sub2_path = 'scratch/test_noninterp_image/sub-P002/sub-P002_desc-rigid_space-T1w_ct.nii.gz'
z_norm2_path = 'scratch/test_noninterp_image/sub-P002/sub-P002_desc-rigid_space-T1w_znorm_ct.nii.gz'

previous_norm = normalization_shifted(test_subject_path)
z_norm = z_normalization(test_subject_path)
min_max = min_max_normalization(test_subject_path)
z_norm2 = z_normalization(sub2_path)

#nib.save(previous_norm, norm_shift_path)
nib.save(z_norm, z_norm_path)
nib.save(z_norm2,z_norm2_path)
#nib.save(min_max, min_max_path)

1164.762289130268
1017.7536923556506


In [12]:
orig_aff, orig_data, orig_header = read_nii_metadata(test_subject_path)

def robust_scaling(nii_path):
    aff, img_data, img_header = read_nii_metadata(nii_path)
    flat_data = img_data.flatten()
    q1 = np.percentile(flat_data, 25)
    q3 = np.percentile(flat_data, 75) 
    scaled_img = (img_data-q1)/q3-q1
    print(q1, q3)
    img_nii = nib.Nifti1Image(scaled_img, aff, img_header)
    return img_nii

robust_img = robust_scaling(test_subject_path)
#nib.save(robust_img, 'scratch/test_noninterp_image/sub-P001/sub-P001_desc-rigid_robust_scaled_space-T1w_ct.nii.gz')

-1004.0 -38.0
